In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
r = requests.get(base_url)

In [4]:
page = BeautifulSoup(r.text, 'html.parser')

In [5]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [6]:
__EVENTVALIDATION

'/wEdABKgcz9IYBHfjBYwXgPnnH/S4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc18rBNWX+/2+gn/p/KZl5VOeeJP6yluj+9GihNPsLwYKFydP60Kwv8njaL32/dcjKGVE3N9enVlXxlsNkU+46/iYUejXo5tHr9D6UzWxMjqKyKwsIJ0B6TbsdMKWnSs3r9tteqB5Dx8umKZ8iT8Tu0RjLmDzA=='

In [7]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [8]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYEHghpZGZHcnVwbwL/////Dx4IYnVzY2FQb3JkFgJmD2QWAgIDD2QWAgIBDxYCHgZhY3Rpb24FFi9kYWRvc2FiZXJ0b3MvY29uc3VsdGEWDgICDxAPFgYeDURhdGFUZXh0RmllbGQFCk5vbWVQYWdpbmEeDkRhdGFWYWx1ZUZpZWxkBQlVcmxQYWdpbmEeC18hRGF0YUJvdW5kZxYEHglvbmtleWRvd24FQ2lmKElzRW50ZXJLZXkoZXZlbnQpKXtSZWRpcmVjaW9uYXJBY2Vzc29SYXBpZG8odGhpcyk7cmV0dXJuIGZhbHNlO30eB29uY2xpY2sFH1JlZGlyZWNpb25hckFjZXNzb1JhcGlkbyh0aGlzKTsQFQYOQWNlc3NvIFLDoXBpZG8WU2VjcmV0YXJpYXMgZSDDk3Jnw6NvcxFTZXJ2acOnb3MgT24tTGluZRhTZXJ2acOnb3MgcGFyYSBhIEVtcHJlc2EZU2VydmnDp29zIHBhcmEgbyBDaWRhZMOjbxNTaXRlcyBkYSBQcmVmZWl0dXJhFQYADS9zZWNyZXRhcmlhcy8ZL2xpc3RhLXNlcnZpY29zLW9uLWxpbmUvMhEvc2Vydmljb3MvZW1wcmVzYREvc2Vydmljb3MvY2lkYWRhbxwvbGlzdGEtc2l0ZXMtZGEtcHJlZmVpdHVyYS81FCsDBmdnZ2dnZ2RkAgQPD2QWAh8GBUlyZXR1cm4gVmFsaWRhclBlc3F1aXNhKCd0eHRQZXNxdWlzYVRvcG8nLCdjdGwwMCRsbmtQZXNxdWlzYVRvcG8nLCBldmVudCk7ZAIGDw8WAh4EVGV4dAUOU8OhYiwgMTMvMTAvMThkZAIHDw8WAh4ISW1hZ2VVcmwFFX4vaW1hZ2VtL3RlbXBvL3BjLmdpZmRkAggPDxYCHwgFBDI1wrpkZAIJDw8WAh8IBQQxNcK6ZGQCCg9kFg4CAw9kFgQCAQ8

In [9]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [10]:
pagination

['1', '2', '3', '4', '5', '6', '7']

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

979df6ebc2280f5c834cc3dff183df24


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [20]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = 'http://'+a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 172/172 [00:00<00:00, 59420.17it/s]


In [21]:
databases

[{'database_id': '979df6ebc2280f5c834cc3dff183df24',
  'database_title': 'Disque Economia - Base de Dados',
  'database_link': 'http://multimidia.curitiba.pr.gov.br/dadosabertos/DisqueEconomia/2018-10-08_Disque_Economia_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 8)},
 {'database_id': '203db94be8f280e2c9911f44dfcf2685',
  'database_title': 'Disque Economia - Produto - Base de Dados',
  'database_link': 'http://multimidia.curitiba.pr.gov.br/dadosabertos/DisqueEconomia/2018-10-08_Disque_Economia_-_Produto_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 8)},
 {'database_id': '66b0019727446b81e6667622e30b2874',
  'database_title': '156 - Base de Dados',
  'database_link': 'http://multimidia.curitiba.pr.gov.br/dadosabertos/156/2018-10-01_156_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)},
 {'database_id': 'fcd48779a785811b55d386a5f9863c10',
  'database_title': '156 - Historico - Base de Dados',
  'database_link': 'http://multimidia

In [26]:
from IPython.display import display, HTML

In [34]:
for database in tqdm.tqdm(databases):
    try:
        data = pd.read_csv(database['database_link'], encoding="ISO-8859-1", delimiter=';', error_bad_lines=False, warn_bad_lines=False)
        head = data.fillna('').head()

        display(HTML(f'<h2>{database["database_title"]}</h2>'))
        display(HTML(head.to_html()))
    except:
        display(HTML(f'<h2 style="color: red">Error on: {database["database_title"]}</h2>'))





  0%|          | 0/92 [00:00<?, ?it/s]

,DATA_PESQUISA,PRECO_PESQUISADO,CODIGO_ESTABELECIMENTO,CODIGO_ESTABELECIMENTO_FILIAL,ENDERECO_ESTABELECIMENTO,NUMERO_ENDERECO_ESTABELECIMENTO,COMPLEMENTO_ENDERECO_ESTABELECIMENTO,TELEFONE_ESTABELECIMENTO,CIDADE_ESTABELECIMENTO,ESTADO_ESTABELECIMENTO,CODIGO_BAIRRO,DESCRICAO_BAIRRO,ESTABELECIMENTO_REDE,ESTABELECIMENTO_RAZAO_SOCIAL,CODIGO_CATEGORIA_PRODUTO,DESCRICAO_CATEGORIA_PRODUTO,CODIGO_PRODUTO,PRODUTO_DESCRICAO,PRODUTO_QTD_EMBALAGEM,CODIGO_TIPO_UNIDADE_MEDIDA_PRODUTO,TIPO_UNIDADE_MEDIDA_PRODUTO_DESCRICAO,TIPO_UNIDADE_MEDIDA_PRODUTO_SIGLA,CODIGO_TIPO_EMBALAGEM_PRODUTO,TIPO_EMBALAGEM_PRODUTO_DESCRICAO,TIPO_EMBALAGEM_PRODUTO_SIGLA
0,-------------,----------------,----------------------,-----------------------------,------------------------,-------------------------------,------------------------------------,------------------------,----------------------,----------------------,-------------,----------------,--------------------,----------------------------,------------------------,---------------------------,--------------,-----------------,---------------------,----------------------------------,-------------------------------------,---------------------------------,-----------------------------,--------------------------------,----------------------------
1,09/07/2018,5.95,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,45,MOSSUNGUE,CONDOR,SUPERMERCADOS CONDOR LTDA.,1,1,2,Adoçante Em Po/ Acucar Light,500.0,2,GRAMA,GR,11,PACOTE,PCT
2,09/07/2018,2.45,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,45,MOSSUNGUE,CONDOR,SUPERMERCADOS CONDOR LTDA.,1,1,4,Acucar Refinado ( + Barato ),1.0,1,KILOGRAMA,KG,11,PACOTE,PCT
3,09/07/2018,7.39,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,45,MOSSUNGUE,CONDOR,SUPERMERCADOS CONDOR LTDA.,1,1,9,"Amido de Milho ""Maizena""",500.0,2,GRAMA,GR,4,CAIXA,CXA
4,09/07/2018,2.11,2,22,RUA MARTIM AFONSO,2800,CHAMPAGNAT,3019-2800,CURITIBA,PR,45,MOSSUNGUE,CONDOR,SUPERMERCADOS CONDOR LTDA.,1,1,11,Farinha de Trigo Especial (+ Barato),1.0,1,KILOGRAMA,KG,11,PACOTE,PCT






  1%|          | 1/92 [00:27<42:22, 27.94s/it]

,CODIGO_PRODUTO,PRODUTO_DESCRICAO,CATEGORIA_PRODUTO,TIPO_EMBALAGEM_PRODUTO_SIGLA,UNIDADE
0,--------------,-----------------,-----------------,----------------------------,-------
1,1,Adocante ( + Barato ),CONFEITARIA E PANIFICACAO,FRA,100ML
2,2,Adoçante Em Po/ Acucar Light,CONFEITARIA E PANIFICACAO,PCT,500GR
3,3,Acucar Cristal ( + Barato ),CONFEITARIA E PANIFICACAO,PCT,5KG
4,4,Acucar Refinado ( + Barato ),CONFEITARIA E PANIFICACAO,PCT,1KG






  2%|▏         | 2/92 [00:28<21:06, 14.07s/it]

...





 98%|█████████▊| 90/92 [09:51<00:13,  6.57s/it]

,UFU_IDF,UFU_NOME,UFU_SITUACAO,UFU_SIGLA,TUF_IDF,COD_REGIONAL,UFU_ATIVO
0,-------,--------,------------,---------,-------,------------,---------
1,1,SGM - ADMINISTRAÇÃO REGIONAL CAJURU,A,SGMARCJ,2,4,
2,2,SGM - ADMINISTRAÇÃO REGIONAL BAIRRO NOVO,A,SGMARBN,2,6,
3,3,SGM - ADMINISTRAÇÃO REGIONAL BOA VISTA,A,SGMARBV,2,3,
4,4,SGM - ADMINISTRAÇÃO REGIONAL BOQUEIRÃO,A,SGMARBQ,2,2,






 99%|█████████▉| 91/92 [09:51<00:06,  6.50s/it]

,UFU_IDF,SET_IDF,UFI_UR,UFI_EXECUTOR
0,-------,-------,------,------------
1,1,5,1,0
2,1,6,1,0
3,1,7,1,0
4,1,8,1,0






100%|██████████| 92/92 [09:51<00:00,  6.43s/it]



